## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-16-10-45-39 +0000,nyt,Live Updates: Trump Backs Putin’s Demand for F...,https://www.nytimes.com/live/2025/08/16/world/...
1,2025-08-16-10-43-26 +0000,nyt,"Arrests in Turkey Silence Erdogan’s Rivals, Op...",https://www.nytimes.com/2025/08/16/world/europ...
2,2025-08-16-10-39-43 +0000,bbc,Trump touts peace deal after leaving Alaska wi...,https://www.bbc.com/news/articles/ce836yz8r69o...
3,2025-08-16-10-36-32 +0000,nyt,Israel Gears Up for Nationwide Strike to Suppo...,https://www.nytimes.com/2025/08/16/world/middl...
4,2025-08-16-10-27-52 +0000,nyt,Russia Holds Battlefield Advantage in Ukraine War,https://www.nytimes.com/live/2025/08/16/world/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2345/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,71
4,putin,43
10,ukraine,20
444,meeting,15
144,new,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
207,2025-08-15-15-56-48 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...,228
232,2025-08-15-13-25-54 +0000,nypost,Putin’s foreign minister trolls Ukraine by wea...,https://nypost.com/2025/08/15/world-news/putin...,205
35,2025-08-16-08-01-18 +0000,nyt,Trump and Putin had an isolated conversation i...,https://www.nytimes.com/live/2025/08/15/world/...,191
100,2025-08-16-00-14-50 +0000,nypost,"Putin tries to woo Trump at Alaska meeting, cl...",https://nypost.com/2025/08/15/us-news/putin-tr...,186
74,2025-08-16-02-30-31 +0000,nypost,Trump’s ‘quiet’ stance after Alaska meeting wi...,https://nypost.com/2025/08/15/us-news/trumps-q...,178


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
207,228,2025-08-15-15-56-48 +0000,nyt,"How Trump’s Meeting With Putin Could Unfold, a...",https://www.nytimes.com/2025/08/15/us/politics...
146,58,2025-08-15-20-58-21 +0000,nypost,Trump reacts to Hillary Clinton saying she’d n...,https://nypost.com/2025/08/15/us-news/trump-re...
60,45,2025-08-16-03-57-56 +0000,nyt,Trump Administration Backs Off New Attempt to ...,https://www.nytimes.com/2025/08/15/us/washingt...
51,43,2025-08-16-05-01-19 +0000,bbc,'Next time in Moscow?': Five takeaways after T...,https://www.bbc.com/news/articles/c4gj9er0x0zo...
91,36,2025-08-16-00-44-32 +0000,latimes,California's first for-sale ADU built in San J...,https://www.latimes.com/california/story/2025-...
47,34,2025-08-16-05-45-17 +0000,nypost,"Air Canada cabin staff go on strike, grounding...",https://nypost.com/2025/08/16/world-news/air-c...
171,31,2025-08-15-19-20-08 +0000,nypost,High-ranking Hamas terrorist killed by IDF ami...,https://nypost.com/2025/08/15/world-news/hamas...
175,28,2025-08-15-18-27-02 +0000,nypost,Feds accuse New Orleans Mayor LaToya Cantrell ...,https://nypost.com/2025/08/15/us-news/mayor-la...
40,27,2025-08-16-06-45-52 +0000,nypost,"Senior, 76, died while trying to meet Meta AI ...",https://nypost.com/2025/08/16/us-news/nj-senio...
81,25,2025-08-16-01-50-10 +0000,bbc,"No ceasefire, no deal: What summit means for T...",https://www.bbc.com/news/articles/clyvd3gkg1po...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
